# سؤالِ پنجم

 در این تمرین یک مقدار با خواص مقدار ویژه‌های ماتریس‌های هرمیتی آشنا می‌شویم.
فرض کنید $A$ یک ماتریس هرمیتی باشد.
 
1. نشان دهید که به ازای هر بردار
$\vert{\psi}\rangle$
عبارت
$\langle \psi \vert A\vert \psi\rangle$
یک عدد حقیقی است.

2. نشان دهید 
$$\lambda_0 = \min_{\vert{\psi}\rangle:\, \|\vert\psi\rangle\|=1} \langle \psi \vert A\vert \psi\rangle$$
یک مقدار
ویژه ماتریس $A$ است.
در واقع اگر $\vert{\psi_0}\rangle$
برداری باشد که عبارت فوق در آن کمینه می‌شود، آنگاه داریم 
$A\vert{\psi_0}\rangle = \lambda_0\vert{\psi_0}\rangle$

راهنمایی: ضرایب لاگرانژ را گوگل کنید و مشتق بگیرید.


3. نشان دهید
$\lambda_0$
کوچکترین‌ مقدار ویژه $A$
است.


4. نشان دهید $\langle \psi \vert A\vert \psi\rangle$
به ازای هر بردار $\vert \psi\rangle$
مثبت است اگر و فقط اگر همه مقدار ویژه‌های $A$
مثبت باشند. 


5. فرض کنید $\vert{\phi_1}\rangle, \dots, \vert{\phi_{d}}\rangle$
بردار‌هایی دلخواه باشند. ماتریس $A$ را ماتریسی $d\times d$ بگیرید که درایه 
 $(i,j)$
برابر $\langle i \vert A\vert j \rangle = \langle \phi_i \vert \phi_j \rangle$
باشد.
نشان دهید $A$ هرمیتی است و همه مقادیر ویژه $A$
نامنفی هستند.

## آماده‌سازی‌ها

In [209]:
import numpy as np

In [210]:
# you can create a random matrix/vector using this function
def complex_rand(*args):
    return np.random.rand(*args) * np.exp(1j * 2 * np.pi * np.random.rand(*args))

In [211]:
def dagger(x : str):
    return np.mat(x).conj().T
# e.g. dagger(np.mat([1 + 1j,0,0]))

In [212]:
def tensor_product(a, b):
    return np.kron(a, b)

In [213]:
# part 2
# optimization
from scipy.optimize import minimize
minimize?

In [214]:
# part 3
np.linalg.eig?

## جوابِ سؤال به شکلِ کد

In [215]:
# part 1

# create Hermitian matrices
# a Hermitian matrix is equal to its conjugate transpose

S_A = complex_rand(4,4)
# S + S_dagger is a hermitian matrix
A = S_A + dagger(S_A)

# a random state 
psi = complex_rand(4,1)

# expectation value
E = dagger(psi) * A * psi
print((abs(E - E.conj()) < 0.001).all()) # E - E* = 2 Im(E)

True


In [216]:
# part 2

ket0 = np.array([[1], [0]])
ket1 = np.array([[0], [1]])

S_A = complex_rand(2,2)
# S + S_dagger is a hermitian matrix
A = S_A + dagger(S_A)

# optimization
def opt(x):
    psi = (x[0] + 1j * x[1]) * ket0 + (x[2] + 1j * x[3]) * ket1
    return complex(np.matmul(dagger(psi), np.matmul(A, psi))).real

def opt_J(x):
    der = np.zeros_like(x)
    der[0] = (2 * x[0] *A[0, 0] + (x[2] + 1j * x[3]) * A[0, 1] + (x[2] - 1j * x[3]) * A[1, 0]).real
    der[1] = (2 * x[1] *A[0, 0] - 1j * (x[2] + 1j * x[3]) * A[0, 1] + 1j * (x[2] - 1j * x[3]) * A[1,0]).real
    der[2] = (2 * x[2] *A[1, 1] + (x[0] + 1j * x[1]) * A[1, 0] + (x[0] - 1j * x[1]) * A[0, 1]).real
    der[3] = (2 * x[3] *A[1, 1] - 1j * (x[0] + 1j * x[1]) * A[1, 0] + 1j * (x[0] - 1j * x[1]) * A[0,1]).real
    return der

def opt_H(x):
    x = np.asarray(x)
    H = np.zeros((4, 4))
    H[0, 0] = H[1, 1] = (2 * A[0,0]).real
    H[2, 2] = H[3, 3] = (2 * A[1,1]).real
    H[0, 1] = H[1, 0] = H[2,3] = H[3,2] = 0
    H[0, 2] = H[2, 0] = (A[0,1] + A[1,0]).real
    H[0, 3] = H[3, 0] = (1j * (A[0,1] - A[1,0])).real
    H[1, 2] = H[2, 1] = (1j * (-A[0,1] + A[1,0])).real
    H[1, 3] = H[3, 1] = (A[0,1] + A[1,0]).real
    return H

# Defining Nonlinear Constraints
def cons_f(x):
    return [x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2]
def cons_J(x):
    return [2*x[0], 2*x[1], 2*x[2], 2*x[3]]
def cons_H(x, v):
     return v[0] * np.diag([2, 2, 2, 2])

from scipy.optimize import NonlinearConstraint 
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H)

x0 = np.array([1.0, 0.0, 0.0, 0.0])
res = minimize(opt, x0, method='trust-constr', jac=opt_J, hess=opt_H, options={'verbose': 1},
                constraints=[nonlinear_constraint])

lambda_0 = opt(res.x)
print(np.linalg.norm(res.x))
psi_opt = (res.x[0] + 1j * res.x[1]) * ket0 + (res.x[2] + 1j * res.x[3]) * ket1

print("lambda_0 = ", lambda_0)
print("psi_0 = ", psi_opt)

print((abs(lambda_0 * psi_opt - np.matmul(A, psi_opt)) < 0.001).all())

`gtol` termination condition is satisfied.
Number of iterations: 7, function evaluations: 5, CG iterations: 8, optimality: 5.32e-09, constraint violation: 0.00e+00, execution time: 0.011 s.
0.9977178514236119
lambda_0 =  -0.857107971046558
psi_0 =  [[0.91207158+1.07425008e-15j]
 [0.37226201+1.58073871e-01j]]
False


In [217]:
# part 3

eigenvalue, eigenvector = np.linalg.eig(A)
print(abs(np.sort(eigenvalue)[0] - lambda_0) < 0.01)

True


In [218]:
# part 4

random_psi = complex_rand(2,1)
print("all of eigenvalues are positive : ", np.sort(eigenvalue)[0] > 0)
print("psi-A-psi is positive : ", complex(np.matmul(dagger(random_psi), np.matmul(A, random_psi))).real > 0)

all of eigenvalues are positive :  False
psi-A-psi is positive :  False


In [219]:
# part 5

n = 4
d = 10
phis = [complex_rand(d, 1) for _ in range(n)]
A = np.zeros((n, n), complex)
for i in range(n):
    for j in range(n):
        A[i,j] = complex(np.matmul(dagger(phis[i]), phis[j]))

eigenvalue, eigenvector = np.linalg.eig(A)
print(np.sort(eigenvalue)[0] > 0, (abs(A - dagger(A)) < 0.0001).all())

True True


## جوابِ سؤال به شکلِ ریاضی
### جواب بخش ۱

برای نشان دادن حقیقی بودن یک عدد کافی است نشان دهیم آن عدد با مزدوج مختلطش برابر است.
$$
\langle{\psi}\vert A \vert{\psi}\rangle^*=\sum_{i,j}\psi_i A^*_{ij}\psi_j^*=\sum_{i,j}\psi_j^* A_{ji}\psi_i=\langle{\psi}\vert A \vert{\psi}\rangle
$$

### جواب بخش ۲

از روش لاگرانژ میدانیم برای کمینه کردن 
$\langle \psi\vert A\vert \psi\rangle$
با قید
$\| \vert{\psi}\rangle\|=1$
باید مشتق تابع 
$f= \langle \psi\vert A\vert \psi\rangle+\lambda \langle \psi\vert\psi\rangle -1$
را برابر صفر قرار دهیم.  برای این کار فرض کنید درایه 
$m$ام از بردار $\psi$ برابر است با
$\psi_m=c_m+ib_m$. 
لذا
\begin{align*}
   f= \sum_{jk} (c_j-ib_j)A_{jk}(c_k+ib_k)-\lambda \Big(\sum_{j}(c_j^2+b_j^2)-1\Big).
\end{align*}    
و داریم
\begin{align*}
&  \frac{ \partial f}{\partial c_m}=\sum_{j}A_{mj}(c_j+ib_j)+\sum_j A_{jm} (c_j-ib_j)-\lambda 2 c_m, \\  &
  \frac{ \partial f}{\partial b_m}=-i\sum_{j}A_{mj}(c_j+ib_j)+i\sum_j A_{jm} (c_j-ib_j)-\lambda 2 b_m.
  \end{align*}
اگر برای هر $m$ قرار دهیم
$\frac{ \partial f}{\partial c_m}=\frac{ \partial f}{\partial b_m}=0$
داریم
$2\lambda(c_m+ib_m)=\sum_j 2A_{mj}(c_j+ib_j).$
پس
$$\lambda\begin{bmatrix}
 \psi_1 \\ 
 \vdots \\ 
 \psi_n
 \end{bmatrix} =A\begin{bmatrix}
 \psi_1 \\ 
 \vdots \\ 
 \psi_n
 \end{bmatrix} $$


### جواب بخش ۳

همانطور که در قسمت قبل ثابت کردیم اگر بردار 
$\psi_0$
برداری باشد که عبارت فوق را کمینه میکند باید در معادله
$\lambda\vert{\psi_0}\rangle=A\vert{\phi_0}\rangle$
 صدق کند. لذا یک بردار ویژه ماتریس
 $A$
 است و کمترین مقدار این عبارت برابر کوچکترین مقدار ویژه است.

### جواب بخش ۴

 با توجه به این که نرم هر بردار 
$\vert\psi\rangle$
 مثبت است و قسمت قبل سوال واضح است. 

### جواب بخش ۵

برای هرمیتی بودن کافی است نشان دهیم
$A_{ij}=A^*_{ji}$.
با توجه به تعریف داریم
$A_{ij}=\langle \phi_i\vert \phi_j\rangle=\langle \phi_j\vert \phi_i\rangle^*=A_{ji}^*$.

 برای نشان دادن نامنفی بودن مقادیر ویژه، برای هر بردار
 $\vert\psi\rangle$  
 داریم
 $$\langle \psi\vert A \vert\psi\rangle =\sum_i \psi_i^* \sum_j \langle \phi_i \vert \phi_j\rangle \psi_j= \Big(\sum_i \psi_i^*\langle{\psi_i}\vert\Big)\Big(\sum_j \psi_j \vert{\phi_j}\rangle\Big) =\Big\| \sum_j \psi_j \vert{\phi_j}\rangle\Big\|^2 \geq 0.$$
  پس طبق قسمت قبل همه مقادیر ویژه مثبت هستند.